# WEB SCRAPING


# IMPORTACION DE LIBRERIAS

In [ ]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io
import requests


# DESCARGA DE HTML

In [10]:


# Seleccionar el recurso a descargar
resource_url = "https://abacosuytec.com/suministros-y-consumibles-empresras/limpieza-higiene-ferreteria-industrial.html"

# Petición para descargar el fichero de Internet
response = requests.get(resource_url)
# Verificamos la respuesta
print("Estado:", response.status_code)


Estado: 200


# CONVERTIR HTML A UN DATAFRAME

In [ ]:
soup = BeautifulSoup(response.text, 'html')
names= soup.find_all(class_= "product-item-link")
spans = soup.find_all(class_= "price")
categorias = soup.find_all(class_= "ox-product-grid__categories")

precios = [span.get_text(strip=True).replace('€','').strip() for span in spans]
nombres = [nombre_producto.get_text(strip=True) for nombre_producto in names][1:31]


categorias=[nombre.get_text(strip=True) for nombre in categorias]

df = pd.DataFrame({
    'Categoria': categorias,
    'Nombre': nombres,
    'Precio': precios
})
df

,Categoria,Nombre,Precio
0,Limpieza e Higiene,BROCHA PLANA DOBLE 33/70 MM,"3,92"
1,Limpieza e Higiene,RODILLO ANTIGOTEO 220X50 3471,"10,43"
2,Limpieza e Higiene,CEPILLO RAICES 1100,"2,40"
3,Limpieza e Higiene,CUBO PLASTICO 6 LITROS,"2,40"
4,Limpieza e Higiene,PINCEL PLANO O REDONDO 2,"0,77"
5,Limpieza e Higiene,CEPILLO FIBRA SIN MANGO 1743*,"4,48"
6,Limpieza e Higiene,"BROCHA PLANA TRIPLE 3""/75 MM","10,70"
7,Limpieza e Higiene,RODILLO LANA 220X40 3310,"17,38"
8,Limpieza e Higiene,CUBO GALA OVAL+ESCURRIDOR 13L,"7,65"
9,Limpieza e Higiene,PINCEL PLANO O REDONDO 6,"1,02"
